Programação concorrente e paralela
===

Programação com threads é um vasto e complexo tópico, principalmente devido à questão de evitar conflitos em recursos compartilhados.  Entretanto, para algumas das situações mais comuns, Python fornece soluções mais simples que encapsulam a complexidade atrás de uma API bem definida.

Neste contexto, estudaremos dois módulos de Python que lidam com programação concorrente ou paralela: `threading` e `concurrent.futures`.

O módulo `threading` é o módulo de mais baixo nível entre os três, dando acesso direto às threads e suas informações, além de objetos auxiliares que permitem a sincronização entre as threads.  Devido a esta variedade de funcionalidade, ele é o módulo mais flexível de ser usado, embora seja o mais complexo também.

Os módulos `multiprocessing` e `concurrent.futures` fornecem uma API mais simplificada e restrita, porém ainda assim muito poderosa.  Estes módulos são, em geral, os mais recomendados de serem usados para programação concorrente e paralela.  É importante notar que estes dois módulos podem usar, para a execução concorrente das tarefas, tanto threads como processos.

# Módulo `threading`

## Classe `Thread`

A classe principal é a Thread, usada para encapsular uma thread.  Seu construtor deve ser chamado com os argumentos passados por nomes.  Entre os eles estão o argumento `target`, contendo uma função que será executada pela thread, e os argumentos `args` e `kwargs`, que é uma tupla e um dicionário, respectivamente, a serem passados para a função `target`.  Os valores-padrão de `args` e `kwargs` são uma tupla e um dicionário vazios, respectivamente.

O objeto Thread criado pode ter a thread associada posta para rodar através do método `start()`.  Este método deve ser chamado uma única vez.  Uma vez que a thread execute, o método `run()` é chamado.  Este método, afora o método `__init__()`, é o único que pode ser reimplementado em uma subclasse de `Thread`.  A implementação-padrão de `run()` simplesmente chama o método `target` passado para o construtor da classe juntamente com os argumentos também informados no construtor.

Um outro método que a classe `Thread` possui é o `join()`.  Quem chamar este método irá bloquear esperando a thread terminar.

In [ ]:
import threading
import time

def target(msg="Que horas são?"):
    print("Iniciando a contagem na nova thread:", end=" ")
    for i in range(10):
        print(i, end=" ")
        time.sleep(1)
    print()
    print("Função posta pra dormir às", time.time())
    time.sleep(3)
    print("Função acordou às", time.time())
    print(msg)

th = threading.Thread(target=target, kwargs={"msg": "Que soninho bom..."})
th.start()
print("Iniciando a contagem na thread principal:", end=" ")
for i in range(10):
    print(i, end=" ")
    time.sleep(1)
print()
print("Esperando pela thread...")
th.join()
print("A thread retornou...")

Observe que tanto a thread principal como a thread secundária criada fazem uso do `print()` ao mesmo tempo.  Isto causa conflitos, uma escrevendo por cima da outra.  Isto é típico de quando diferentes threads fazem uso de recursos compartilhados sem sincronia.

Para evitar isso, podemos usar algumas funcionalidades do módulo `threading` como por exemplo, a classe `Lock`.

In [ ]:
import threading
import time

def target(lck, msg="Que horas são?"):
    with lck:
        print("Iniciando a contagem na nova thread:", end=" ")
        for i in range(10):
            print(i, end=" ")
            time.sleep(0.5)
        print()
        print("Função posta pra dormir às", time.time())
    time.sleep(3)
    with lck:
        print("Função acordou às", time.time())
        print(msg)

lck = threading.Lock()

th = threading.Thread(target=target, args=(lck,), kwargs={"msg": "Que soninho bom..."})
th.start()
with lck:
    print("Iniciando a contagem na thread principal:", end=" ")
    for i in range(10):
        print(i, end=" ")
        time.sleep(0.5)
    print()
with lck:
    print("Esperando pela thread...")
th.join()
print("A thread retornou...")

# Módulo `concurrent.futures`

## Classe `ThreadPoolExecutor`

Esta classe permite executar tarefas em threads separadas de forma natural e eficiente.

In [ ]:
import concurrent.futures
import urllib.request

URLS = ['http://www.foxnews.com/',
        'http://www.cnn.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://some-made-up-domain.com/']

# Retrieve a single page and report the URL and contents
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))